# Computer Vision Nanodegree

## Project: Image Captioning

In this notebook, some template code has already been provided for you ...

---

### Introduction

In this notebook, you will xyz.

Feel free to use the links below to navigate the notebook:

(insert table of contents here)

### Step 1: Explore the Dataset

- look at sample images and captions.
- tell how data was preprocessed & saved in cocoapi (256 on shortest side)
- print some statistics (number of images)

### Step 2: Examine the Data Loader

- note to self: RETRAIN WITHOUT PAD WORD ASSIGNMENT
- show how vocab file is generated
- guide their play with `vocab` output
- tell should only amend the threshold ... vocab does have other args that should stay at default values. mention `vocab_from_file`
- used now to prepare for embedding layer
- size of vocab also determines dimensionality of output layer of LSTM & will be used to decode predicted captions. more on that soon
- uncomment one of the lines below to examine word2idx, idx2word. will use word2idx to convert images to nums, idx2word later to convert predictions to words ...
- show how to get a batch of data
- describe how sampler works
- note allowed to change data loader, but ok if leave as-is. to pass project, just need to make sure that you can defend your data loader

In [1]:
from vocabulary import Vocabulary
vocab = Vocabulary(4, vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!


In [4]:
from data_loader import get_loader
from torchvision import transforms

# specify vocab threshold
vocab_threshold = 4

# image preprocessing
transform_train = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# specify batch size
batch_size = 10

data_loader = get_loader(transform=transform_train,
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.47s)
creating index...


  0%|          | 1015/414113 [00:00<00:40, 10140.41it/s]

index created!
Obtaining caption lengths ...


100%|██████████| 414113/414113 [00:41<00:00, 10030.80it/s]


- when train model later, note don't need to separately populate data loader and vocab. vocab built in to data loader ...
- check now that data_loader includes dataset, which includes vocab ... can also get word2idx and idx2word as above.

In [3]:
print('Number of words in vocabulary:', len(data_loader.dataset.vocab))
#print(data_loader.dataset.vocab.word2idx)
#print(data_loader.dataset.vocab.idx2word)

Number of words in vocabulary: 9955


- describe how get_train_indices works, where we got the procedure for loading batches from

In [ ]:
import numpy as np
import torch.utils.data as data

indices = data_loader.dataset.get_train_indices()
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler

for batch in data_loader:
    images, captions = batch[0], batch[1]
    break
    
print('sampled indices:', data_loader.batch_sampler.sampler.indices)
print('images.shape:',images.shape)
print('captions.shape:', captions.shape)

### Step 4: Tinker with the CNN Encoder

- provide them a CNN encoder.
- let them decide if they want to swap out the pre-trained architecture for another (make clear this is allowed). show how to do for vgg-16
- make note of module autoreload capability
- encourage them to stop here and think about dimensionality of output
- need to explain to them what `volatile` is for, etc
- consider adding batch normalization if you like ...

In [ ]:
%load_ext autoreload
% autoreload 2

In [ ]:
import torch
from torch.autograd import Variable
from model import EncoderCNN

# amend this if you like
embed_size = 256

encoder = EncoderCNN(embed_size)
if torch.cuda.is_available():
    encoder = encoder.cuda()

In [ ]:
def to_var(x, volatile=False):
    """ converts a Pytorch Tensor to a variable and moves to GPU if CUDA is available """
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

images_var = to_var(images, volatile=True)
features = encoder(images_var)

print('features.shape:', features.shape)

### Step 5: Tinker with RNN Decoder

- they have to write the decoder here and I plan to provide a unit test where they can check if implementation is correct
- they have to decide hidden size, embed size - won't provide default values but will provide papers they can read
- may need to use vocabulary.  note `len(vocab)` returns the total number of tokens
- if leave notebook and come back, re-run every cell.

In [ ]:
from model import DecoderRNN

embed_size = 256
hidden_size = 512
vocab_size = len(vocab)

decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
if torch.cuda.is_available():
    decoder = decoder.cuda()

In [ ]:
captions_var = to_var(captions)
outputs = decoder(features, captions_var)

print('outputs.shape:', outputs.shape)

### Step 6: Calculate the Loss

- i give the criterion. students have to calculate the loss

In [ ]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO: 
loss = criterion(outputs.view(-1, vocab_size), captions_var.view(-1))

print('loss:', loss.data[0])

---

### Congratulations!

If you have completed all of the steps of this notebook, you're ready to move on to the next notebook in the project sequence.

When you are done with this notebook, please navigate to **1_Train.ipynb**, where you will train your own CNN-RNN model to generate image captions!